# # CS262 Exercise 04 - Generate actionable SAP knowledge summaries with Generative AI

## Introduction

With the help of this Jupyter notebook you will understand how to interact with the self-service Answer Generation API.

## Pre-requisites

Before you proceed, ensure that you have the following libraries installed:

In [ ]:
%pip install aiohttp

---

## Import Libraries

We'll need some Python libraries to make HTTP requests and handle JSON data.

In [ ]:
import json
import aiohttp

## Calling the Self-Service Answer Generation API

The below function performs the HTTP POST request to the Answer Generation API.

In [ ]:
async def fetch_answer(data: dict[str, str]):
    # Define HTTP endpoint to be called
    btp_destination_name = "Cloud_ALM_API.dest"
    url = f"https://{btp_destination_name}/http/supportcases/recommendations/solutions/rag"
    events = []

    # Perform the HTTP request
    async with aiohttp.ClientSession(trust_env=True) as session:
        async with session.post(
            url=url,
            headers={"Content-Type": "application/json", "Accept": "text/event-stream"},
            json=data,
        ) as resp:
            async for chunk, _ in resp.content.iter_chunks():
                events.append(chunk.decode("utf-8"))

    return "".join(events)

In [ ]:
# INPUT PARAMETERS
subject = ""  # Fill with a short problem description
description = ""  # Fill with a detailed problem description
component = "" # Fill with an SAP software component

# API CALL
response = await fetch_answer(
    {
        "subject": subject,
        "description": description,
        "component": component
    }
)

### Inspect the API response

#### Display raw response

In [ ]:
print(response)

#### Parse the response into a Dict for better usability

In [ ]:
parsed_events = []
event = {}
for line in response.splitlines():
    if line:
        parts = line.split(":", 1)
        event[parts[0]] = json.loads(parts[1])
    else:
        parsed_events.append(event)
        event = {}

#### Extract the generated answer

In [ ]:
text = ""
for event in parsed_events:
    if event["event"] == "message":
        text += event["data"]

print(text)